# Supplementary Jupyter notebook for PH 821 project

### Project title: The Effect of Gravitational Waves on a Ring of Particles

Presented by: Aditya Pawan (23D1069)
____________________________________

In this notebook, I will demonstrate how to simulate the effects of gravitational waves on a ring of particles. We will first look at a two-dimensional representation of the ring of particles, then simulate the response of a monochromatic gravitational wave with the polarization amplitudes being interactable.

Later, we shall move to a three-dimensional representation, which allows us to simulate the effect of these gravitational waves originating from a compact binary source with arbitrary position and binary inclination with respect to the ring frame. 

All effects are amplified for better visualization.

Disclaimer: If any animations or widgets break/start lagging, please restart the kernel and run the relevant cells again.

In [ ]:
# Dependencies
"""
Please check if the following modules are installed in your environment:
numpy v1.26.4
matplotlib v3.9.1
mpl_toolkits
"""

import numpy as np
import matplotlib.pyplot as pl

In [ ]:
"""
These specific submodules, functions and classes will be used in this project to produce interactive, animated plots.
"""
from mpl_toolkits import mplot3d
import matplotlib.figure as figure
import matplotlib.axes as axes
from matplotlib.animation import FuncAnimation
import mpl_toolkits.axes_grid1
import matplotlib.widgets as widgets

In [ ]:
"""
Matplotlib widget is required for the more interactive plots that contain matplotlib.widget.Slider and 
matplotlib.widget.Button, as well as FuncAnimation codes. If you wish to switch to a different backend, you may
wish to enter your backend of choice. If you wish to use the default (inline) backend for non-interactive and
non-animated plots, do not run this cell.
"""
%matplotlib widget

## Part 1: Simulating a Two-Dimensional Ring of particles
___

Let us start with a basic plot containing a ring of n = 8 particles.

The cell below will be where our simulation will happen, so move back to it after running the cells below it.

In [ ]:
fig, ax = pl.subplots()
pl.axis('scaled')

n = 8
theta = np.arange(0,2*np.pi,2*np.pi/n)
x = np.cos(theta)
y = np.sin(theta)
sc, = ax.plot(x, y, marker = 'o', ls = "")
ax.grid(True)
ax.set_xlim(-4.0, 4.0)
ax.set_ylim(-4.0, 4.0)
pl.show()

Let us now add two axes which contain sliders that enable us to vary both plus and cross polarizations.

In [ ]:
fig.subplots_adjust(bottom=0.3)

axplus = fig.add_axes([0.35, 0.2, 0.55, 0.03])
hplus = widgets.Slider(ax=axplus,label='Plus Polarization Amplitude',valmin=0.0,valmax=1.0,valinit=0.5)

axcross = fig.add_axes([0.35, 0.1, 0.55, 0.03])
hcross = widgets.Slider(ax=axcross,label='Cross Polarization Amplitude',valmin=0.0,valmax=1.0,valinit=0.5)

def pos(val):
    hp = hplus.val
    hc = hcross.val

hplus.on_changed(pos)
hcross.on_changed(pos)

We would not be able to visualize anything on this plot currently since to do that we would need to see it in motion. Fortunately, Matplotlib's animation sub-module contains the class FuncAnimation that allows us to animate a figure using an update function.

However, the ability to manipulate the passage of time cannot be changed while the code is running. To provide more control to the user, I introduce the "Player" class, which calls the FuncAnimation function and automatically adds an axis that has a slider and buttons for time manipulation.

In [ ]:
class Player(FuncAnimation):
    def __init__(self, fig, func, frames = None, init_func = None, fargs = None, save_count = None, mini = 0, maxi = 100, pos = (0.125, 0.92), cache_frame_data = False, **kwargs):
        self.i = 0
        self.min = mini
        self.max = maxi
        self.runs = True
        self.forwards = True
        self.looped = False
        self.fig = fig
        self.func = func
        self.setup(pos)
        FuncAnimation.__init__(self, self.fig, self.update, frames=self.play(), init_func=init_func, fargs=fargs, save_count=save_count, cache_frame_data=cache_frame_data, **kwargs)
    
    def play(self):
        while self.runs:
            self.i = self.i + self.forwards - (not self.forwards)
            if self.i > self.min and self.i < self.max:
                yield self.i
            else:
                self.stop()
                yield self.i
    
    def start(self):
        self.runs = True
        self.event_source.start()
    
    def stop(self, event=None):
        if not self.looped:
            self.runs = False
            self.event_source.stop()
        else:
            self.forwards = not self.forwards
    
    def forward(self, event=None):
        self.forwards = True
        self.start()
    
    def backward(self, event=None):
        self.forwards = False
        self.start()
    
    def oneforward(self, event=None):
        self.forwards = True
        self.onestep()
    
    def onebackward(self, event=None):
        self.forwards = False
        self.onestep()

    def loop(self, event=None):
        self.looped = not self.looped
    
    def onestep(self):
        if self.i > self.min and self.i < self.max:
            self.i = self.i + self.forwards - (not self.forwards)
        elif self.i == self.min and self.forwards:
            self.i += 1
        elif self.i == self.max and not self.forwards:
            self.i -= 1
        self.func(self.i)
        self.slider.set_val(self.i)
        self.fig.canvas.draw_idle()
    
    def setup(self, pos):
        playerax = self.fig.add_axes([pos[0], pos[1], 0.64, 0.04])
        divider = mpl_toolkits.axes_grid1.make_axes_locatable(playerax)
        bax      = divider.append_axes("right", size="80%",  pad=0.05)
        sax      = divider.append_axes("right", size="80%",  pad=0.05)
        fax      = divider.append_axes("right", size="80%",  pad=0.05)
        ofax     = divider.append_axes("right", size="100%", pad=0.05)
        rax = divider.append_axes("right", size="80%", pad=0.05)
        sliderax = divider.append_axes("right", size="500%", pad=0.07)
        
        
        self.button_oneback    = widgets.Button(playerax,  label="$\u29CF$")
        self.button_back       = widgets.Button(bax,       label="$\u25C0$")
        self.button_stop       = widgets.Button(sax,       label="$\u25A0$")
        self.button_forward    = widgets.Button(fax,       label="$\u25B6$")
        self.button_oneforward = widgets.Button(ofax,      label="$\u29D0$")
        self.button_loop       = widgets.Button(rax,       label='$\u27F3$')
        
        self.button_oneback.on_clicked(self.onebackward)
        self.button_back.on_clicked(self.backward)
        self.button_stop.on_clicked(self.stop)
        self.button_forward.on_clicked(self.forward)
        self.button_oneforward.on_clicked(self.oneforward)
        self.button_loop.on_clicked(self.loop)

        self.slider = widgets.Slider(sliderax, '', self.min, self.max, valinit=self.i)

        self.slider.on_changed(self.set_pos)
    
    def set_pos(self, i):
        self.i = int(self.slider.val)
        self.func(self.i)

    def update(self, i):
        self.slider.set_val(i)

Now with the aforementioned _Player_ class in place, we construct an update function for it to update our figure continuously (or as the time slider changes).

Run the cell below and go back to our plot to simulate Gravitational Waves!

In [ ]:
def pos_ret_XY(val):
    t = np.linspace(0,20,102)
    hp = hplus.val*np.sin(2*np.pi*t/20)
    hc = hcross.val*np.sin(2*np.pi*t/20)
    dx = np.array([np.real(0.5*hp*np.cos(theta[i]) + 0.5*hc*np.sin(theta[i])) for i in range(len(theta))])
    dy = np.array([np.real(-0.5*hp*np.sin(theta[i]) + 0.5*hc*np.cos(theta[i])) for i in range(len(theta))])
    X = np.zeros((len(t), len(theta)))
    Y = np.zeros((len(t), len(theta)))
    X[0], Y[0] = x, y

    for i in range(val + 1):
        X[i+1], Y[i+1] = x + dx[:,i], y + dy[:,i]
    return X, Y


def update(i):
    X, Y = pos_ret_XY(i)
    xt = X[i+1]
    yt = Y[i+1]
    sc.set_data(xt, yt)

ani = Player(fig, update, maxi=100, interval = 60)

## Part 2: Simulating the effect of a compact binary source in Three Dimensions
___

Now that we have seen how gravitational waves affect a ring of particles propagated perpendicular to the plane where the ring sits, the next step is to of course ask how the same gravitational wave's effect will affect the ring when the wave is propagating from an arbitrary direction.

Not only that, but as we are simulating compact binary sources, we would like to see how the deviation of the orbit normal to the source vector (which we will refer to as the "_polarization_" angle) affects what is seen in the ring.

In [ ]:
"""
In case you needed to restart the kernel for running part 2, here are all the necessary package imports.
"""
import numpy as np
import matplotlib.pyplot as pl
from mpl_toolkits import mplot3d
import matplotlib.figure as figure
import matplotlib.axes as axes
from matplotlib.animation import FuncAnimation
import mpl_toolkits.axes_grid1
import matplotlib.widgets as widgets
%matplotlib widget

Now, in order to simulate this effect in 3D space, first we need a good representation of the orientation of the binary source.

The binary source is located at an arbitrary point _P_ $(r \sin(\theta)\cos(\phi), r \sin(\theta)\sin(\phi), r \cos(\theta))$. But that's not all.

Imagine that the normal to this plane, that points in the opposite direction of $\vec{OP}$ where _O_ is the origin, is now rotated by an angle $\psi$ about the normal direction. This is the polarization angle, and it is the third Euler angle that allows for us to convert our gravitational wave amplitude from the source frame to the ring frame using the Antenna functions $F_{+}(\theta, \phi, \psi)$ and $F_{\times}(\theta, \phi, \psi)$

Running the cells below will generate a 3D plot where the angles can be varied accordingly.

In [ ]:
def plot_projection(theta, phi, psi): # now with an added circle with polarization angle
    figure = pl.figure()

    axes = figure.add_axes([0,0.3,1,0.7],projection='3d')
    pl.axis('scaled')
    radial = np.linspace(0, 10, 101)
    xline = radial*np.sin(theta)*np.cos(phi)
    yline = radial*np.sin(theta)*np.sin(phi)
    zline = radial*np.cos(theta)

    alpha = np.linspace(0,2*np.pi,101)
    xloc, yloc, zloc = xline[100], yline[100], zline[100]
    xcircle = xloc - 2*np.cos(alpha)*np.cos(psi)*np.sin(phi) + 2*np.cos(alpha)*np.sin(psi)*np.sin(theta)*np.cos(phi) + 2*np.sin(alpha)*np.cos(theta)*np.cos(phi)
    ycircle = yloc + 2*np.cos(alpha)*np.cos(psi)*np.cos(phi) + 2*np.cos(alpha)*np.sin(psi)*np.sin(theta)*np.sin(phi) + 2*np.sin(alpha)*np.cos(theta)*np.sin(phi)
    zcircle = zloc + 2*np.cos(alpha)*np.sin(psi)*np.cos(theta) - 2*np.sin(alpha)*np.sin(theta)

    xvec = -5*(np.cos(psi)*np.sin(theta)*np.cos(phi) + np.sin(psi)*np.sin(phi))
    yvec = -5*(np.cos(psi)*np.sin(theta)*np.sin(phi) - np.sin(psi)*np.cos(phi))
    zvec = -5*np.cos(psi)*np.cos(theta)

    plot1, = axes.plot(xline, yline, zline,'b')
    plot2, = axes.plot(xcircle, ycircle, zcircle, 'g')
    plot3 = axes.quiver(xloc, yloc, zloc, xvec, yvec, zvec, color='purple', alpha=0.8, lw=3)
    axes.set_xlim(-10,10)
    axes.set_ylim(-10,10)
    axes.set_zlim(-10,10)

    return figure, axes, plot1, plot2, plot3

In [ ]:
fig, ax, p1, p2, p3 = plot_projection(np.pi/2, np.pi, 0.0)

In [ ]:
axtheta = fig.add_axes([0.2, 0.2, 0.65, 0.03])
axphi = fig.add_axes([0.2, 0.1, 0.65, 0.03])
axpsi = fig.add_axes([0.2, 0.0, 0.65, 0.03])
resetax = fig.add_axes([0.2, 0.3, 0.1, 0.03])
theta = widgets.Slider(axtheta, 'theta (in pi units)', valmin=0.0, valmax=1.0, valinit=0.5)
phi = widgets.Slider(axphi, 'phi (in pi units)', valmin=0.0, valmax=2.0, valinit=1.0)
psi = widgets.Slider(axpsi, 'psi (in pi units)', valmin=-1.0, valmax=1.0, valinit=0.0)
reset_button = widgets.Button(resetax, 'Reset', hovercolor='0.975')

In [ ]:
def set_theta_phi(i):
    global ax, p1, p2, p3
    radial = np.linspace(0, 10, 101)
    th = theta.val*np.pi
    ph = phi.val*np.pi
    ps = psi.val*np.pi
    xline = radial*np.sin(th)*np.cos(ph)
    yline = radial*np.sin(th)*np.sin(ph)
    zline = radial*np.cos(th)
    
    alpha = np.linspace(0,2*np.pi,101)
    xloc, yloc, zloc = xline[100], yline[100], zline[100]
        
    xcircle = xloc - 2*np.cos(alpha)*np.cos(ps)*np.sin(ph) + 2*np.cos(alpha)*np.sin(ps)*np.sin(th)*np.cos(ph) + 2*np.sin(alpha)*np.cos(th)*np.cos(ph)
    ycircle = yloc + 2*np.cos(alpha)*np.cos(ps)*np.cos(ph) + 2*np.cos(alpha)*np.sin(ps)*np.sin(th)*np.sin(ph) + 2*np.sin(alpha)*np.cos(th)*np.sin(ph)
    zcircle = zloc + 2*np.cos(alpha)*np.sin(ps)*np.cos(th) - 2*np.sin(alpha)*np.sin(th)

    xvec = -5*(np.cos(ps)*np.sin(th)*np.cos(ph) + np.sin(ps)*np.sin(ph))
    yvec = -5*(np.cos(ps)*np.sin(th)*np.sin(ph) - np.sin(ps)*np.cos(ph))
    zvec = -5*np.cos(ps)*np.cos(th)

    p1.set_data(xline, yline)
    p1.set_3d_properties(zline)
    p2.set_data(xcircle, ycircle)
    p2.set_3d_properties(zcircle)
    p3.remove()
    p3 = ax.quiver(xloc, yloc, zloc, xvec, yvec, zvec, color='purple', alpha=0.8, lw=3)
    kwall = i
    ax.set_xlim(-10,10)
    ax.set_ylim(-10,10)
    ax.set_zlim(-10,10)

In [ ]:
def reset(i):
    theta.reset()
    phi.reset()
    psi.reset()

reset_button.on_clicked(reset)

theta.on_changed(set_theta_phi)
phi.on_changed(set_theta_phi)
psi.on_changed(set_theta_phi)

Now that we have successfully made our representation of the compact binary with respect to the ring located near the origin, we turn our attention to creating and animating the ring.

Run the cells below to start the 3D simulation!

In [ ]:
beta = np.arange(0,2*np.pi,np.pi/4)
x = 5*np.cos(beta)
y = 5*np.sin(beta)
z = 5*np.zeros(np.size(beta))
sc, = ax.plot(x, y, z, linestyle="", marker='o')
fig.subplots_adjust(bottom=0.3, left=0.3)

In [ ]:
axplus = fig.add_axes([0.05, 0.3, 0.03, 0.55])
hplus = widgets.Slider(ax=axplus,label='Plus',valmin=0.0,valmax=5.0,valinit=2.5,orientation='vertical')

axcross = fig.add_axes([0.15, 0.3, 0.03, 0.55])
hcross = widgets.Slider(ax=axcross,label='Cross',valmin=0.0,valmax=5.0,valinit=2.5,orientation='vertical')

In [ ]:
def reset(i):
    hplus.reset()
    hcross.reset()
    theta.reset()
    phi.reset()
    psi.reset()

In [ ]:
reset_button.on_clicked(reset)

In [ ]:
def pos(val):
    hp = hplus.val
    hc = hcross.val

hplus.on_changed(pos)
hcross.on_changed(pos)

In [ ]:
class Player(FuncAnimation):
    def __init__(self, fig, func, frames = None, init_func = None, fargs = None, save_count = None, mini = 0, maxi = 100, pos = (0.125, 0.92), cache_frame_data = False, **kwargs):
        self.i = 0
        self.min = mini
        self.max = maxi
        self.runs = True
        self.forwards = True
        self.looped = False
        self.fig = fig
        self.func = func
        self.setup(pos)
        FuncAnimation.__init__(self, self.fig, self.update, frames=self.play(), init_func=init_func, fargs=fargs, save_count=save_count, cache_frame_data=cache_frame_data, **kwargs)
    
    def play(self):
        while self.runs:
            self.i = self.i + self.forwards - (not self.forwards)
            if self.i > self.min and self.i < self.max:
                yield self.i
            else:
                self.stop()
                yield self.i
    
    def start(self):
        self.runs = True
        self.event_source.start()
    
    def stop(self, event=None):
        if not self.looped:
            self.runs = False
            self.event_source.stop()
        else:
            self.forwards = not self.forwards
    
    def forward(self, event=None):
        self.forwards = True
        self.start()
    
    def backward(self, event=None):
        self.forwards = False
        self.start()
    
    def oneforward(self, event=None):
        self.forwards = True
        self.onestep()
    
    def onebackward(self, event=None):
        self.forwards = False
        self.onestep()

    def loop(self, event=None):
        self.looped = not self.looped
    
    def onestep(self):
        if self.i > self.min and self.i < self.max:
            self.i = self.i + self.forwards - (not self.forwards)
        elif self.i == self.min and self.forwards:
            self.i += 1
        elif self.i == self.max and not self.forwards:
            self.i -= 1
        self.func(self.i)
        self.slider.set_val(self.i)
        self.fig.canvas.draw_idle()
    
    def setup(self, pos):
        playerax = self.fig.add_axes([pos[0], pos[1], 0.64, 0.04])
        divider = mpl_toolkits.axes_grid1.make_axes_locatable(playerax)
        bax      = divider.append_axes("right", size="80%",  pad=0.05)
        sax      = divider.append_axes("right", size="80%",  pad=0.05)
        fax      = divider.append_axes("right", size="80%",  pad=0.05)
        ofax     = divider.append_axes("right", size="100%", pad=0.05)
        rax = divider.append_axes("right", size="80%", pad=0.05)
        sliderax = divider.append_axes("right", size="500%", pad=0.07)
        
        
        self.button_oneback    = widgets.Button(playerax,  label="$\u29CF$")
        self.button_back       = widgets.Button(bax,       label="$\u25C0$")
        self.button_stop       = widgets.Button(sax,       label="$\u25A0$")
        self.button_forward    = widgets.Button(fax,       label="$\u25B6$")
        self.button_oneforward = widgets.Button(ofax,      label="$\u29D0$")
        self.button_loop       = widgets.Button(rax,       label='$\u27F3$')
        
        self.button_oneback.on_clicked(self.onebackward)
        self.button_back.on_clicked(self.backward)
        self.button_stop.on_clicked(self.stop)
        self.button_forward.on_clicked(self.forward)
        self.button_oneforward.on_clicked(self.oneforward)
        self.button_loop.on_clicked(self.loop)

        self.slider = widgets.Slider(sliderax, '', self.min, self.max, valinit=self.i)

        self.slider.on_changed(self.set_pos)
    
    def set_pos(self, i):
        self.i = int(self.slider.val)
        self.func(self.i)

    def update(self, i):
        self.slider.set_val(i)

In [ ]:
def wave2detframe(hp,hc,theta,phi,psi):
    Fp = 0.5*(1 + np.cos(theta)**2)*np.cos(2*phi)*np.cos(2*psi) - np.cos(theta)*np.sin(2*phi)*np.sin(2*psi)
    Fc = 0.5*(1 + np.cos(theta)**2)*np.cos(2*phi)*np.sin(2*psi) + np.cos(theta)*np.sin(2*phi)*np.cos(2*psi)

    return hp*Fp, hc*Fc

In [ ]:
def pos_ret_XY(val):
    t = np.linspace(0,20,102)
    hp, hc = wave2detframe(hplus.val, hcross.val, theta.val, phi.val, psi.val)

    hpp = hp*np.cos(2*np.pi*t/20)
    hcc = hc*np.cos(2*np.pi*t/20)

    dx = np.array([np.real(0.5*hpp*np.cos(beta[i]) + 0.5*hcc*np.sin(beta[i])) for i in range(len(beta))])
    dy = np.array([np.real(-0.5*hpp*np.sin(beta[i]) + 0.5*hcc*np.cos(beta[i])) for i in range(len(beta))])
    X = np.zeros((len(t), len(beta)))
    Y = np.zeros((len(t), len(beta)))
    X[0], Y[0] = x, y

    for i in range(val + 1):
        X[i+1], Y[i+1] = x + dx[:,i], y + dy[:,i]
    return X, Y

In [ ]:
def update(i):
    X, Y = pos_ret_XY(i)
    zt = 0
    xt = X[i+1]
    yt = Y[i+1]
    sc.set_data(xt, yt)
    sc.set_3d_properties(zt)

ani = Player(fig, update, maxi=100, interval = 60)

You have reached the end of this supplementary Jupyter notebook. 

Thank you for your time, and hope you enjoyed running and playing with this simulation as much as I did!

Comments are much appreciated.